<a href="https://colab.research.google.com/github/Athugodage/for_inca/blob/main/choose_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Создание датасета

In [ ]:
!pip install google-play-scraper

In [ ]:
crypto_apps = {'co.mona.android' : "Crypto.com - Buy BTC, ETH",
               'com.kucoin.wallet': 'KuCoin Wallet: Crypto & NFT',
               'com.coinbase.android': 'Coinbase: Buy Bitcoin & Ether',
               'com.kraken.invest.app': 'Kraken - Buy Bitcoin & Crypto',
               'com.robinhood.android': 'Robinhood: Stocks & Crypto',
               'com.wallet.crypto.trustapp': 'Trust: Crypto & Bitcoin Wallet',
               'com.coindcx.btc': 'CoinDCX:Bitcoin Investment App',
               'com.btcw.bitcoinwallet': 'SoBit: Bitcoin Wallet - BTC',
               'com.phemex.app': 'Phemex: Buy Crypto & Bitcoin',
               'com.binance.dev': 'Binance: BTC, 600+ Crypto, NFT'
               }

CSV-файл со следующей структурой:app name username timestamp app review text sentiment score country (language) marketplace*

In [ ]:
import csv
from google_play_scraper import Sort, reviews_all, app
import numpy as np
import pandas as pd

class CryptoReviews():
    def __init__(self, apps):
        self.apps = apps

    def new_dataset(self, name):
        with open(str(name), 'w', newline='') as file: 
            writer = csv.writer(file)
            writer.writerow(['App name', 'Username', 'Timestamp', 'App review text', 'Sentiment score',
                             'country (language)', 'Marketplace'])
            
    def _scrape(self, app):
        result = reviews_all(
                    app,
                    sleep_milliseconds=1, 
                    lang='en', 
                    sort=Sort.MOST_RELEVANT, 
                    filter_score_with=5)
        return result


    def collect(self, name, limit):
        self.limit = limit
        self.personal_limit = int(self.limit / len(self.apps.keys()))
        self.n = 0

        with open(name, 'a', newline='') as file:
            writer = csv.writer(file)
            
            for n, app in enumerate(self.apps):
                result = self._scrape(app)

                for review in result[0:self.personal_limit]:
                    name = self.apps[app]
                    user = review['userName'] # username
                    time = review['at'] # timestamp
                    text = review['content'] # text
                    score = np.nan # sentiment score
                    lang = np.nan # language
                    market = 'Google Play Store'
                    writer.writerow([name, user, time, text, score, lang, market])
                print(n)                                 
                        

                
                    


In [ ]:
c = CryptoReviews(crypto_apps)

In [ ]:
c.collect('for_inca.csv', 1000)

In [ ]:
df = pd.read_csv('for_inca.csv')
df.groupby('App name').agg({'App name': 'count'})

In [ ]:
df

# we have a little more than 1000 reviews because I was collecting them in two steps

**I. Chechking different methods**


We will try 3 different methods. The one that will have the highest score will be used for our data.

1.Sentiment Analysis with Transformers

In [21]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 27.8 MB/s 
     |████████████████████████████████| 6.6 MB 61.0 MB/s 
     |████████████████████████████████| 120 kB 55.5 MB/s 


In [24]:
!pip install torchtext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd

df = pd.read_csv('for_inca.csv')

In [19]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

stopword = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [20]:
from nltk.corpus import movie_reviews

nltk.download('movie_reviews')

pos_raw= movie_reviews.raw(movie_reviews.fileids('pos')) 
neg_raw= movie_reviews.raw(movie_reviews.fileids('neg')) 

pos= pos_raw.split('\n')   
neg= pos_raw.split('\n')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


In [21]:
pos_tag= []       # giving each sentence label 
neg_tag= []
for i in pos:
    pos_tag.append([i,1])
for i in neg:
    neg_tag.append([i,0])

In [23]:
from sklearn.utils import shuffle
import pandas as pd

pos= pd.DataFrame(pos_tag,columns=['txt','status'])
neg= pd.DataFrame(neg_tag,columns=['txt','status'])

data= pd.concat([pos,neg],axis=0)     
data=shuffle(data)

In [8]:
print('Total amount: ', len(data))

Total amount:  65876


In [9]:
data.groupby('status').agg('count')

,txt
status,
0,32938
1,32938


In [31]:
from transformers import pipeline

classifier = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")

# we use pre-trained model

https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpsn1wzcq9


storing https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/cb09766f7ba60b5f7a1bb640617b24f1499c4a6f3ab160c4a0ac171e3a377c68.008dca06003188334001a96363da79ced4944abc68d94a2f1e0db786dc5aa08b
creating metadata file for /root/.cache/huggingface/transformers/cb09766f7ba60b5f7a1bb640617b24f1499c4a6f3ab160c4a0ac171e3a377c68.008dca06003188334001a96363da79ced4944abc68d94a2f1e0db786dc5aa08b
loading configuration file https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/cb09766f7ba60b5f7a1bb640617b24f1499c4a6f3ab160c4a0ac171e3a377c68.008dca06003188334001a96363da79ced4944abc68d94a2f1e0db786dc5aa08b
Model config RobertaConfig {
  "_name_or_path": "finiteautomata/bertweet-base-sentiment-analysis",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_

Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

loading configuration file https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/cb09766f7ba60b5f7a1bb640617b24f1499c4a6f3ab160c4a0ac171e3a377c68.008dca06003188334001a96363da79ced4944abc68d94a2f1e0db786dc5aa08b
Model config RobertaConfig {
  "_name_or_path": "finiteautomata/bertweet-base-sentiment-analysis",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "NEG",
    "1": "NEU",
    "2": "POS"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "NEG": 0,
    "NEU": 1,
    "POS": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "n

storing https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/2e4719cf8d097772eb75070b88cbc56f1d3b1392fffc5f75032a389ef21d1847.16366ca1277caccb15200478349503b3336a1420ac26d44fc16763354f5a2cae
creating metadata file for /root/.cache/huggingface/transformers/2e4719cf8d097772eb75070b88cbc56f1d3b1392fffc5f75032a389ef21d1847.16366ca1277caccb15200478349503b3336a1420ac26d44fc16763354f5a2cae
loading weights file https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/2e4719cf8d097772eb75070b88cbc56f1d3b1392fffc5f75032a389ef21d1847.16366ca1277caccb15200478349503b3336a1420ac26d44fc16763354f5a2cae
All model checkpoint weights were used when initializing RobertaForSequenceClassification.

All the weights of RobertaForSequenceClassification were initialized from the model checkpoint at finiteautomata/bertw

storing https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/61374b71c02fdfd2929a3cdce24c242049e036624e15e18461a3a70cfc35e939.c260b44e952f7f2a825aac395f2ebbed4ac9553800d1e320af246e81a548f37c
creating metadata file for /root/.cache/huggingface/transformers/61374b71c02fdfd2929a3cdce24c242049e036624e15e18461a3a70cfc35e939.c260b44e952f7f2a825aac395f2ebbed4ac9553800d1e320af246e81a548f37c
loading configuration file https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/cb09766f7ba60b5f7a1bb640617b24f1499c4a6f3ab160c4a0ac171e3a377c68.008dca06003188334001a96363da79ced4944abc68d94a2f1e0db786dc5aa08b
Model config RobertaConfig {
  "_name_or_path": "finiteautomata/bertweet-base-sentiment-analysis",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "

storing https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/973dbacfdf4c488622f01d1a226089e9e3dba130a0c3c11c2e36d49466fa40a8.f8a4dfe5c3c45a26f9df849d732decb191dc0c05ab270799695430332d143982
creating metadata file for /root/.cache/huggingface/transformers/973dbacfdf4c488622f01d1a226089e9e3dba130a0c3c11c2e36d49466fa40a8.f8a4dfe5c3c45a26f9df849d732decb191dc0c05ab270799695430332d143982
https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis/resolve/main/bpe.codes not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpl2rpnz6v


storing https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis/resolve/main/bpe.codes in cache at /root/.cache/huggingface/transformers/0e474c44ff353f3b378fb140e7e6d4431df4ec6142e8b38d584c0dbc5afc3521.75877d86011e5d5d46614d3a21757b705e9d20ed45a019805d25159b4837b0a4
creating metadata file for /root/.cache/huggingface/transformers/0e474c44ff353f3b378fb140e7e6d4431df4ec6142e8b38d584c0dbc5afc3521.75877d86011e5d5d46614d3a21757b705e9d20ed45a019805d25159b4837b0a4
https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis/resolve/main/added_tokens.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp747fr622


storing https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis/resolve/main/added_tokens.json in cache at /root/.cache/huggingface/transformers/fe46927817477a58ec2aa92ef52f8ee6fc9e824d054f4aa6a3c129724dc9c9b7.c1e7052e39d2135302ec27455f6db22e1520e6539942ff60a849c7f83f8ec6dc
creating metadata file for /root/.cache/huggingface/transformers/fe46927817477a58ec2aa92ef52f8ee6fc9e824d054f4aa6a3c129724dc9c9b7.c1e7052e39d2135302ec27455f6db22e1520e6539942ff60a849c7f83f8ec6dc
https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpa5u3_3eg


storing https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/9413ac0bed76140860deffa0c5a29ee4da7d49a3810da1b4b51b27f790bc9255.0dc5b1041f62041ebbd23b1297f2f573769d5c97d8b7c28180ec86b8f6185aa8
creating metadata file for /root/.cache/huggingface/transformers/9413ac0bed76140860deffa0c5a29ee4da7d49a3810da1b4b51b27f790bc9255.0dc5b1041f62041ebbd23b1297f2f573769d5c97d8b7c28180ec86b8f6185aa8
loading file https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/973dbacfdf4c488622f01d1a226089e9e3dba130a0c3c11c2e36d49466fa40a8.f8a4dfe5c3c45a26f9df849d732decb191dc0c05ab270799695430332d143982
loading file https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis/resolve/main/bpe.codes from cache at /root/.cache/huggingface/transformers/0e474c44ff353f3b378fb140e7e6d4431df4ec6142e8b38d584c0dbc5afc3

In [34]:
preds = classifier(data['txt'].tolist()[:300])

# classifing the whole corpus takes a lot of time. Here we just evaluate different methods - so the first random 300 is enough

In [36]:
import numpy as np

def tag2num(preds):
    d = []
    for label in preds:
        if label['label'] == 'NEG':
            d.append(0)
        elif label['label'] == 'POS':
            d.append(1)
        else:
            d.append(1)
    return d

pred_nums = np.array(tag2num(preds))

# movie corpus has only 2 classes(pos, neg) so we need a result with only 2 classes

In [37]:
pred_nums

array([0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1])

In [39]:
DT_roc = roc_auc_score(pred_nums, data.status.to_list()[:300])
DT_f1 = f1_score(pred_nums, data.status.to_list()[:300])

print('roc_auc_score:  ', DT_roc)
print('f1_score:  ', DT_f1)

roc_auc_score:   0.4575845879876963
f1_score:   0.5434173669467788


A fine-tuned model

In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 28.0 MB/s 
     |████████████████████████████████| 212 kB 22.9 MB/s 
     |████████████████████████████████| 120 kB 51.8 MB/s 
     |████████████████████████████████| 115 kB 51.9 MB/s 
     |████████████████████████████████| 127 kB 81.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
# Load data
from datasets import load_dataset
imdb = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
# Create a smaller training dataset for faster training times
small_train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(3000))])
small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(300))])
print(small_train_dataset[0])
print(small_test_dataset[0])

{'text': 'There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...', 'label': 1}
{'text': "<br /><br />When I unsuspectedly rented A Thousand Acres, I thought I was in for an entertaining King Lear story and of course Michelle Pfeiffer was in it, so what could go wrong?<br /><br />Very quickly, 

In [5]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 28.8 MB/s 
     |████████████████████████████████| 6.6 MB 63.4 MB/s 


In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [7]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

In [10]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
    load_f1 = load_metric("f1")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"f1": f1}

In [11]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [13]:
from transformers import TrainingArguments, Trainer

repo_name = "finetuning-sentiment-model-3000-samples"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch", 
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Cloning https://huggingface.co/marcus2000/finetuning-sentiment-model-3000-samples into local empty directory.


In [14]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3000
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 376


Step,Training Loss


Saving model checkpoint to finetuning-sentiment-model-3000-samples/checkpoint-188
Configuration saved in finetuning-sentiment-model-3000-samples/checkpoint-188/config.json
Model weights saved in finetuning-sentiment-model-3000-samples/checkpoint-188/pytorch_model.bin
tokenizer config file saved in finetuning-sentiment-model-3000-samples/checkpoint-188/tokenizer_config.json
Special tokens file saved in finetuning-sentiment-model-3000-samples/checkpoint-188/special_tokens_map.json
tokenizer config file saved in finetuning-sentiment-model-3000-samples/tokenizer_config.json
Special tokens file saved in finetuning-sentiment-model-3000-samples/special_tokens_map.json
Saving model checkpoint to finetuning-sentiment-model-3000-samples/checkpoint-376
Configuration saved in finetuning-sentiment-model-3000-samples/checkpoint-376/config.json
Model weights saved in finetuning-sentiment-model-3000-samples/checkpoint-376/pytorch_model.bin
tokenizer config file saved in finetuning-sentiment-model-3000

TrainOutput(global_step=376, training_loss=0.30971604205192405, metrics={'train_runtime': 284.3667, 'train_samples_per_second': 21.1, 'train_steps_per_second': 1.322, 'total_flos': 783875831546880.0, 'train_loss': 0.30971604205192405, 'epoch': 2.0})

In [16]:
eval_transformers = trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 300
  Batch size = 16



2.Sentiment Analysis with TF-IDF (sklearn) 

2.1.Multinomial Naive Bayes



In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer as tfid
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score

In [24]:
vectorizer= tfid(stop_words= stopword, max_features=21)
X = vectorizer.fit_transform(data['txt'])
y = data.status

In [25]:
X_train, X_test, y_train, y_test= train_test_split(X, y, random_state= 42)

In [26]:
from sklearn.naive_bayes import MultinomialNB        # using multiNomial Naive Bayes as classifier

clf= MultinomialNB()

clf.fit(X_train, y_train)

pred= clf.predict(X_test)

MNB_roc = roc_auc_score(pred, y_test)
MNB_f1 = f1_score(pred, y_test)

print('roc_auc_score:  ', MNB_roc)
print('f1_score:  ', MNB_f1)


roc_auc_score:   0.4860456924095868
f1_score:   0.3021248339973439


2.2 Logistic Regression

In [27]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)

pred = lr.predict(X_test)

LR_roc = roc_auc_score(pred, y_test)
LR_f1 = f1_score(pred, y_test)

print('roc_auc_score:  ', LR_roc)
print('f1_score:  ', LR_f1)

roc_auc_score:   0.4886271510250746
f1_score:   0.3900174114915844


II. Comparing methods

In [55]:
comparison = pd.DataFrame({'Pipeline': ['Logistic Regression', 'MNaive Bayes', 'finiteautomata', 'distilbert'],
             'F1': [LR_f1,  MNB_f1, DT_f1, eval_transformers['eval_f1']],
              'ROC': [LR_roc, MNB_roc, DT_roc, np.nan]})

comparison

,Pipeline,F1,ROC
0,Logistic Regression,0.390017,0.488627
1,MNaive Bayes,0.302125,0.486046
2,finiteautomata,0.543417,0.457585
3,distilbert,0.868421,NaN


So, distilbert (transformers fine-tuned model) is the best and we will use it for our data. See the second notebook
